# 🎮 CarRacing AI 테스트 노트북

학습된 AI의 주행을 테스트하고 영상으로 확인합니다.

**전제 조건**: `colab_train.ipynb`를 먼저 실행하여 모델을 학습시켜야 합니다.

---

## 1. Google Drive 마운트

학습된 모델을 Google Drive에서 불러옵니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

# Google Drive 경로 설정
DRIVE_BASE_PATH = '/content/drive/MyDrive/CarRacing_AI'

print("✅ Google Drive 마운트 완료!")
print(f"📁 모델 경로: {DRIVE_BASE_PATH}")

## 2. 필요한 패키지 설치

**중요:** 각 Colab 노트북은 독립된 세션이므로 패키지를 다시 설치해야 합니다.

In [ ]:
# SWIG 먼저 설치 (Box2D 빌드에 필요)
!apt-get update -qq
!apt-get install -y swig build-essential python3-dev

# Box2D 먼저 설치
!pip install box2d-py

# 나머지 패키지 설치
!pip install gymnasium stable-baselines3[extra] shimmy moviepy pygame -q

print("✅ 패키지 설치 완료!")

## 3. 라이브러리 임포트

In [ ]:
import os
import glob
import gymnasium as gym
from stable_baselines3 import PPO
from gymnasium.wrappers import RecordVideo
from IPython.display import HTML, display
from base64 import b64encode

print("✅ 라이브러리 임포트 완료!")

## 4. 설정 (Google Drive 경로)

In [ ]:
# Google Drive에서 모델 경로 설정
MODEL_PATH = f"{DRIVE_BASE_PATH}/models/final_car_racing_driver"
VIDEO_FOLDER = "./video_output"  # 비디오는 로컬에 임시 저장
NUM_EPISODES = 3  # 테스트할 에피소드 수

# 디렉토리 생성
os.makedirs(VIDEO_FOLDER, exist_ok=True)

print("✅ 설정 완료!")
print(f"📁 모델 경로: {MODEL_PATH}")

## 5. 학습된 모델 로드

In [ ]:
print("🤖 학습된 모델 로딩 중...")

try:
    model = PPO.load(MODEL_PATH)
    print("✅ 모델 로드 완료!")
except FileNotFoundError:
    print(f"❌ 모델 파일을 찾을 수 없습니다: {MODEL_PATH}")
    print("먼저 colab_train.ipynb를 실행하여 모델을 학습시켜주세요.")
    print(f"\n또는 Google Drive에서 다음 경로를 확인하세요:")
    print(f"   MyDrive/CarRacing_AI/models/")
    raise

## 6. 비디오 녹화 환경 설정

테스트 환경을 생성하고 비디오 녹화를 설정합니다.

In [ ]:
print("🎥 비디오 녹화 환경 설정 중...")

# 비디오 녹화 환경 생성
eval_env = gym.make("CarRacing-v3", continuous=True, render_mode="rgb_array")
eval_env = RecordVideo(
    eval_env,
    video_folder=VIDEO_FOLDER,
    episode_trigger=lambda x: True,  # 모든 에피소드 녹화
    name_prefix="test-episode"
)

print("✅ 환경 설정 완료!")

## 7. 주행 테스트 실행

이제 학습된 AI가 실제로 운전하는 모습을 테스트합니다!

**예상 점수:**
- 100k 체크포인트: +200~+300점
- Best 모델: 최고 성능

In [ ]:
print("="*50)
print("🏁 주행 테스트 시작!")
print("="*50)

for episode in range(NUM_EPISODES):
    print(f"\n📍 에피소드 {episode + 1}/{NUM_EPISODES} 시작...")
    
    obs, _ = eval_env.reset()
    done = False
    total_reward = 0
    steps = 0
    
    while not done:
        # 모델이 행동 예측
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = eval_env.step(action)
        
        total_reward += reward
        steps += 1
        done = terminated or truncated
    
    print(f"🏁 에피소드 {episode + 1} 종료!")
    print(f"   총 스텝: {steps}")
    print(f"   획득 점수: {total_reward:.2f}")

# 환경 종료
eval_env.close()
print("\n✅ 테스트 완료!")

## 8. 저장된 비디오 확인

In [ ]:
print("="*50)
print("📹 저장된 비디오 파일:")
print("="*50)

video_files = sorted(glob.glob(f'{VIDEO_FOLDER}/*.mp4'))
for i, video_file in enumerate(video_files, 1):
    file_size = os.path.getsize(video_file) / (1024 * 1024)  # MB
    print(f"{i}. {os.path.basename(video_file)} ({file_size:.2f} MB)")

print(f"\n✅ 총 {len(video_files)}개의 비디오가 '{VIDEO_FOLDER}' 폴더에 저장되었습니다!")

## 9. 비디오 재생 (Colab 내에서)

저장된 영상을 Colab 노트북에서 바로 재생합니다.

In [ ]:
def show_video(video_path, width=600):
    """
    Colab에서 비디오를 재생하는 함수
    """
    mp4 = open(video_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f"""
    <video width={width} controls>
        <source src="{data_url}" type="video/mp4">
    </video>
    """)

# 첫 번째 비디오 재생
if video_files:
    print(f"🎬 첫 번째 에피소드 재생: {os.path.basename(video_files[0])}")
    display(show_video(video_files[0]))
else:
    print("❌ 재생할 비디오가 없습니다.")

## 10. 모든 에피소드 재생

In [ ]:
from google.colab import files

# 모든 비디오 다운로드
for video_file in video_files:
    files.download(video_file)
    print(f"📥 다운로드: {os.path.basename(video_file)}")

print("\n✅ 모든 비디오 다운로드 완료!")

## 11. 단일 에피소드 테스트 (선택사항)

녹화 없이 빠르게 한 에피소드만 테스트해볼 수 있습니다.

In [ ]:
# 녹화 없이 빠른 테스트
test_env = gym.make("CarRacing-v3", continuous=True)

obs, _ = test_env.reset()
done = False
total_reward = 0
steps = 0

print("🏎️ 빠른 테스트 시작...")

while not done:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, _ = test_env.step(action)
    total_reward += reward
    steps += 1
    done = terminated or truncated

test_env.close()

print(f"✅ 테스트 완료!")
print(f"   총 스텝: {steps}")
print(f"   획득 점수: {total_reward:.2f}")

---

## 결과 분석

### 점수 해석:
- **음수 (-100 ~ 0)**: AI가 제대로 운전하지 못함
- **0 ~ 200**: 기본적인 운전 가능, 트랙을 조금씩 따라감
- **200 ~ 500**: 트랙을 잘 따라가며 안정적인 주행
- **500 ~ 900+**: 완주 또는 거의 완주 수준

### 성능이 낮다면:
1. 학습 시간을 늘려보세요 (TOTAL_TIMESTEPS 증가)
2. 체크포인트 중 가장 좋은 모델을 사용해보세요
3. 하이퍼파라미터를 조정해보세요

### 성공!
축하합니다! 여러분의 AI가 스스로 운전하는 법을 배웠습니다! 🎉
